In [1]:
import PIL
from PIL import Image
import numpy as np
import tensorflow as tf
import keras
from keras.models import Sequential
from keras.layers import Input,LSTM, Dense, Dropout, Conv2D, MaxPool2D, Flatten, Activation,  MaxPooling2D, BatchNormalization
from keras import regularizers, metrics

In [2]:
def img_t_train_data(img_name, droprate = 0, truncate=True):
    image = Image.open(img_name)
    image = image.convert('RGB')
    # print(image.format)
    # print(image.size)
    # print(image.mode)
    data = np.asarray(image)
    img_width = data.shape[0]
    img_height = data.shape[1]
    data = data.astype("float64")/255
    input1 = np.zeros(((img_height-3)*(img_width-3),15,3))
    input2 = np.zeros(((img_height-3)*(img_width-3),2))
    target = np.zeros(((img_height-3)*(img_width-3),3))
    input_i = 0;
    print(data.shape, input1.shape)
    for i in range(3,img_width):
        for j in range(3,img_height):
            input2[input_i,0] = i/img_width
            input2[input_i,1] = j/img_height
            input1[input_i,0]=data[i-3][j-3]
            input1[input_i,1]=data[i-2][j-3]
            input1[input_i,2]=data[i-1][j-3]
            input1[input_i,3]=data[i][j-3]
            input1[input_i,4]=data[i-3][j-2]
            input1[input_i,5]=data[i-2][j-2]
            input1[input_i,6]=data[i-1][j-2]
            input1[input_i,7]=data[i][j-2]
            input1[input_i,8]=data[i-3][j-1]
            input1[input_i,9]=data[i-2][j-1]
            input1[input_i,10]=data[i-1][j-1]
            input1[input_i,11]=data[i][j-1]
            input1[input_i,12]=data[i-3][j]
            input1[input_i,13]=data[i-2][j]
            input1[input_i,14]=data[i-1][j]
            target[input_i] = data[i][j]
            input_i +=1
    # manually reduce the sample size of exactly same pixels as preceding only keeps 10% of uniform case
    delete_list = []
    r = np.random.rand(input1.shape[0])
    for i in range(input1.shape[0]):
        if (input1[i] == input1[i][0]).all() and r[i] < droprate:
            delete_list.append(i)

    input1 = np.delete(input1,delete_list,0)
    input1_flatten = input1.reshape((input1.shape[0],15*3))
    input2 = np.delete(input2,delete_list,0)
    input = np.concatenate((input1_flatten,input2),axis=1)
    target = np.delete(target,delete_list,0)
    input = input.reshape((input.shape[0],47,1))
    if truncate:
        incompletebatch = input.shape[0] % 100
        delete_list = range(input.shape[0]-incompletebatch, input.shape[0])
        input = np.delete(input,delete_list,0)
        target = np.delete(target,delete_list,0)
    print(input.shape, target.shape)
    return (input, target)

In [3]:
inputs = []
targets = []

for i in range(1,31):
    input,target = img_t_train_data("train"+str(i)+".png")
    inputs.append(input)
    targets.append(target)


(600, 600, 3) (356409, 15, 3)
(356400, 47, 1) (356400, 3)
(600, 600, 3) (356409, 15, 3)
(356400, 47, 1) (356400, 3)
(600, 600, 3) (356409, 15, 3)
(356400, 47, 1) (356400, 3)
(600, 600, 3) (356409, 15, 3)
(356400, 47, 1) (356400, 3)
(600, 600, 3) (356409, 15, 3)
(356400, 47, 1) (356400, 3)
(600, 600, 3) (356409, 15, 3)
(356400, 47, 1) (356400, 3)
(600, 600, 3) (356409, 15, 3)
(356400, 47, 1) (356400, 3)
(329, 329, 3) (106276, 15, 3)
(106200, 47, 1) (106200, 3)
(329, 329, 3) (106276, 15, 3)
(106200, 47, 1) (106200, 3)
(600, 600, 3) (356409, 15, 3)
(356400, 47, 1) (356400, 3)
(600, 600, 3) (356409, 15, 3)
(356400, 47, 1) (356400, 3)
(600, 600, 3) (356409, 15, 3)
(356400, 47, 1) (356400, 3)
(600, 600, 3) (356409, 15, 3)
(356400, 47, 1) (356400, 3)
(600, 600, 3) (356409, 15, 3)
(356400, 47, 1) (356400, 3)
(600, 600, 3) (356409, 15, 3)
(356400, 47, 1) (356400, 3)
(600, 600, 3) (356409, 15, 3)
(356400, 47, 1) (356400, 3)
(600, 600, 3) (356409, 15, 3)
(356400, 47, 1) (356400, 3)
(600, 600, 3) 

In [4]:
model = Sequential()
model.add(keras.layers.InputLayer(input_shape=(47,1)))
# model.add(LSTM(256,stateful=True))
model.add(Flatten())
model.add(Dense(15*16*3, activation="tanh"))
model.add(Dropout(0.1))
model.add(Dense(15*16*3, activation="tanh"))
model.add(Dropout(0.1))
model.add(Dense(5*4*3, activation="relu"))
model.add(Dense(3, activation="relu"))
model.compile(loss=tf.keras.losses.MeanAbsoluteError(), metrics=[metrics.MeanAbsoluteError()], optimizer='adam')

In [5]:
model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 flatten (Flatten)           (None, 47)                0         
                                                                 
 dense (Dense)               (None, 720)               34560     
                                                                 
 dropout (Dropout)           (None, 720)               0         
                                                                 
 dense_1 (Dense)             (None, 720)               519120    
                                                                 
 dropout_1 (Dropout)         (None, 720)               0         
                                                                 
 dense_2 (Dense)             (None, 60)                43260     
                                                                 
 dense_3 (Dense)             (None, 3)                 1

In [6]:
valid_in, valid_target = img_t_train_data("test2.png",droprate=0)
for j in range(1):
    for i in range(0, len(inputs)):
        print(i)
        model.fit(inputs[i], targets[i], batch_size=200,epochs=1, validation_data=(valid_in, valid_target))
        model.reset_states()

(600, 600, 3) (356409, 15, 3)
(356400, 47, 1) (356400, 3)
0
1782/1782 [==============================] - 10s 4ms/step - loss: 0.2659 - mean_absolute_error: 0.2659 - val_loss: 0.0422 - val_mean_absolute_error: 0.0422
1
1782/1782 [==============================] - 7s 4ms/step - loss: 0.0184 - mean_absolute_error: 0.0184 - val_loss: 0.0384 - val_mean_absolute_error: 0.0384
2
1782/1782 [==============================] - 7s 4ms/step - loss: 0.0193 - mean_absolute_error: 0.0193 - val_loss: 0.0283 - val_mean_absolute_error: 0.0283
3
1782/1782 [==============================] - 7s 4ms/step - loss: 0.0138 - mean_absolute_error: 0.0138 - val_loss: 0.0270 - val_mean_absolute_error: 0.0270
4
1782/1782 [==============================] - 7s 4ms/step - loss: 0.0296 - mean_absolute_error: 0.0296 - val_loss: 0.0356 - val_mean_absolute_error: 0.0356
5
1782/1782 [==============================] - 8s 4ms/step - loss: 0.0115 - mean_absolute_error: 0.0115 - val_loss: 0.0391 - val_mean_absolute_error: 0.0391

In [7]:
# predictmodel = Sequential()
# predictmodel.add(keras.layers.InputLayer(batch_input_shape=(1,47,1)))
# predictmodel.add(LSTM(256,stateful=True))
# predictmodel.add(Dense(15*16*3, activation="tanh"))
# predictmodel.add(Dropout(0.1))
# predictmodel.add(Dense(15*16*3, activation="tanh"))
# predictmodel.add(Dropout(0.1))
# predictmodel.add(Dense(5*4*3, activation="relu"))
# predictmodel.add(Dense(3, activation="relu"))
# predictmodel.compile(loss=tf.keras.losses.MeanAbsoluteError(), metrics=[metrics.MeanAbsoluteError()], optimizer='adam')
# predictmodel.set_weights(model.get_weights())

In [16]:
#  return the residual image
# def predict_img(img_name, model):
predictmodel = model
img_name = "test2.png"
image = Image.open(img_name)
image = image.convert('RGB')
data = np.asarray(image)
img_width = data.shape[0]
img_height = data.shape[1]
data = data.astype("float64")/255
img_input,target = img_t_train_data(img_name,droprate=0,truncate=False)
model.reset_states()
result = predictmodel.predict(img_input, batch_size=100)
print (result.shape)

# return residual

(600, 600, 3) (356409, 15, 3)
(356409, 47, 1) (356409, 3)
(356409, 3)


In [17]:
result = result.reshape((img_width-3,img_height-3,3))
resultint = result*255
resultint = np.rint(resultint)
resultint.clip(0,255)
data = np.asarray(image).astype("float64")
residual = data[3:,3:] - resultint
print(np.average(np.absolute(residual)))

9.568297470976697


In [18]:
data[3:,3:] = residual
datachunk = np.rint(data).astype("int")


In [19]:
with open("datachunk.npy", 'wb') as f:
    np.save(f,datachunk)


In [20]:
with open("datachunk.npy", 'rb') as f:
    a = np.load(f)

In [21]:
from huffman import huffman 
# total_bits = int(huffman(np.asarray(image).astype("float64").flatten()))
total_bits = int(huffman(datachunk.flatten()))
print('total: ', total_bits)
result2 = result

 Char | Huffman code 
----------------------
 0    |          01
freq:  (0, 154574)
 9    |          11
freq:  (9, 124842)
 10   |         001
freq:  (10, 113445)
 3    |        0000
freq:  (3, 83794)
 8    |        1011
freq:  (8, 68889)
 2    |        1000
freq:  (2, 65227)
 1    |        1001
freq:  (1, 54758)
 22   |       00010
freq:  (22, 51368)
 -12  |      101010
freq:  (-12, 33025)
 4    |      101001
freq:  (4, 28760)
 11   |     1010110
freq:  (11, 25103)
 12   |     1010111
freq:  (12, 24407)
 7    |     1010001
freq:  (7, 22447)
 20   |    00011110
freq:  (20, 20341)
 -3   |    00011111
freq:  (-3, 17632)
 -9   |    00011100
freq:  (-9, 16371)
 -2   |    00011101
freq:  (-2, 12549)
 5    |    00011010
freq:  (5, 7519)
 -8   |    00011000
freq:  (-8, 6980)
 21   |    00011001
freq:  (21, 6523)
 -4   |    10100000
freq:  (-4, 6415)
 -1   |    10100001
freq:  (-1, 5584)
 -7   |   000110110
freq:  (-7, 5414)
 6    |00011011111111110
freq:  (6, 5288)
 44   |00011011111111111
fr

In [22]:
#  return the residual image
# def predict_img(img_name, model):
img_name = "test1.png"
image = Image.open(img_name)
image = image.convert('RGB')
data = np.asarray(image)
img_width = data.shape[0]
img_height = data.shape[1]
data = data.astype("float64")/255
img_input,target = img_t_train_data(img_name,droprate=0,truncate=False)
model.reset_states()
result = predictmodel.predict(img_input, batch_size=100)
print (result.shape)
result = result.reshape((img_width-3,img_height-3,3))
resultint = result*255
resultint = np.rint(resultint)
resultint.clip(0,255)
data = np.asarray(image).astype("float64")
residual = data[3:,3:] - resultint
print(np.average(np.absolute(residual)))
data[3:,3:] = residual
datachunk = np.rint(data).astype("int")
total_bits = int(huffman(datachunk.flatten()))
print('total: ', total_bits)
result1 = result

(600, 600, 3) (356409, 15, 3)
(356409, 47, 1) (356409, 3)
(356409, 3)
11.326644388890292
 Char | Huffman code 
----------------------
 11   |         011
freq:  (11, 100580)
 17   |         111
freq:  (17, 77141)
 10   |        0100
freq:  (10, 77110)
 9    |        0101
freq:  (9, 75413)
 3    |        0010
freq:  (3, 73193)
 18   |        0011
freq:  (18, 71119)
 19   |        0000
freq:  (19, 69862)
 5    |        0001
freq:  (5, 64374)
 4    |        1100
freq:  (4, 58729)
 2    |        1101
freq:  (2, 49484)
 16   |        1010
freq:  (16, 43818)
 8    |        1011
freq:  (8, 42411)
 -6   |        1001
freq:  (-6, 40718)
 0    |       10001
freq:  (0, 31758)
 15   |    10000110
freq:  (15, 26113)
 14   |    10000111
freq:  (14, 16164)
 12   |    10000100
freq:  (12, 15896)
 6    |    10000101
freq:  (6, 15242)
 7    |    10000010
freq:  (7, 13101)
 1    |    10000011
freq:  (1, 8107)
 21   |    10000000
freq:  (21, 7748)
 -5   |   100000010
freq:  (-5, 5509)
 -3   |1000000111111

In [23]:
#  return the residual image
# def predict_img(img_name, model):
img_name = "test3.png"
image = Image.open(img_name)
image = image.convert('RGB')
data = np.asarray(image)
img_width = data.shape[0]
img_height = data.shape[1]
data = data.astype("float64")/255
img_input,target = img_t_train_data(img_name,droprate=0,truncate=False)
model.reset_states()
result = predictmodel.predict(img_input, batch_size=100)
print (result.shape)
result = result.reshape((img_width-3,img_height-3,3))
resultint = result*255
resultint = np.rint(resultint)
resultint.clip(0,255)
data = np.asarray(image).astype("float64")
residual = data[3:,3:] - resultint
print(np.average(np.absolute(residual)))
data[3:,3:] = residual
datachunk = np.rint(data).astype("int")
total_bits = int(huffman(datachunk.flatten()))
print('total: ', total_bits)
result3 = result

(600, 600, 3) (356409, 15, 3)
(356409, 47, 1) (356409, 3)
(356409, 3)
20.025103182018412
 Char | Huffman code 
----------------------
 0    |           1
freq:  (0, 208688)
 43   |         010
freq:  (43, 134956)
 26   |        0110
freq:  (26, 107217)
 11   |        0111
freq:  (11, 82572)
 10   |        0010
freq:  (10, 73449)
 42   |        0000
freq:  (42, 71646)
 27   |       00110
freq:  (27, 53289)
 25   |       00111
freq:  (25, 52303)
 9    |       00011
freq:  (9, 46202)
 8    |      000101
freq:  (8, 27478)
 1    |   000100110
freq:  (1, 16852)
 5    |   000100111
freq:  (5, 15524)
 2    |   000100100
freq:  (2, 13486)
 3    |  0001001010
freq:  (3, 11084)
 4    |  0001000110
freq:  (4, 10447)
 6    |  0001000111
freq:  (6, 10273)
 21   |  0001000100
freq:  (21, 8771)
 30   |  0001000101
freq:  (30, 8490)
 124  |  0001000010
freq:  (124, 7202)
 -3   |  0001000011
freq:  (-3, 6625)
 -2   |  0001000000
freq:  (-2, 5483)
 7    |  0001000001
freq:  (7, 5424)
 255  |0001001011111